In [ ]:
%pip install -r requirements.txt

Create a Payi client and budget

In [ ]:
from payi import Payi
import os 

#Read the API KEYs from the environment, replace the default values (the second argument) with your own keys if needed
openai_key = os.getenv("OPENAI_API_KEY", "YOUR_OPENAI_KEY")
payi_api_key = os.getenv("PAYI_API_KEY", "YOUR_PAYI_API_KEY")

payi_client = Payi(
    api_key=payi_api_key
)

#Create a budget
budget_response = payi_client.budgets.create(
    #As long as the budget configuration remains the same across creates, the same budget name can be used repeatedly
    budget_name='Langchain quickstart liberal allow budget', 
    max=12.50, #$12.50 USD
    budget_type="Liberal",
    budget_response_type="Allow",
    budget_tags=["example_budget"]
)

budget_name = budget_response.budget.budget_name
budget_id = budget_response.budget.budget_id

print("Budget Created")
print(f"Budget Name: {budget_name}")
print(f"Budget ID: {budget_id}")



Define the callback to be invoked when the LLM call ends

In [ ]:
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate

class PayiHandler(BaseCallbackHandler):
    def __init__(self, client, params):
        self.name = "custom_handler"
        self.client = client
        self.params = {
            **params
        }

    def on_llm_end(self, response, **kwargs):
        llm_output = response.llm_output
        if llm_output and 'token_usage' in llm_output:
            token_usage = llm_output['token_usage']
            prompt_tokens = token_usage.get('prompt_tokens', 0)
            completion_tokens = token_usage.get('completion_tokens', 0)

            if not (prompt_tokens > 0 or completion_tokens > 0):
                print(f"{self.name}: no token usage in LLM output", response)
                return

            try:
                # uncomment either budget_ids or request_tags if you want to use them
                result = self.client.ingest.units(
                    category=self.params['category'],
                    resource=self.params['resource'],
                    input=prompt_tokens,
                    output=completion_tokens,
                    budget_ids=self.params['budget_ids'], 
                    request_tags=self.params['request_tags']
                    )
                print(f'ingest result: {result.model_dump_json(indent=4)}')
            except Exception as e:
                print(f"{self.name}: error sending usage info", e)

Configure the parameters for the call to OpenAI and ingesting the token counts into Payi

In [ ]:
# Configuration parameters for the Pay-i API
params = {
    'category': 'system.openai',
    'resource': 'gpt-3.5-turbo',
    'budget_ids': [budget_id],  
    'request_tags': ['x-ingest', 'y-ingest']
}

Make an OpenAI request and register the callback handler

In [ ]:
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(["say this only: hi"])

# Define the handler
handler = PayiHandler(client=payi_client, params=params)

# Define the LLM and register the handler 
model = ChatOpenAI( 
    model=params['resource'],
    api_key=openai_key,
    callbacks=[handler]
    )

# Define the sequence
chain = prompt | model

# Run the sequence
response = chain.invoke({})

print(response.to_json())